In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)


In [ ]:
model_id = "distilbert-base-uncased"
dataset_id = "ag_news"


In [ ]:
dataset = load_dataset(dataset_id)

train_dataset = dataset['train']
val_dataset = dataset['test'].shard(num_shards=2, index=0)
test_dataset = dataset['test'].shard(num_shards=2, index=1)


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_id)


In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(model_id, num_labels=4)


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:
trainer.train()


In [ ]:
trainer.evaluate()


In [ ]:
trainer.save_model("./saved_model")
